In [20]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from typing import Type, Optional
import os
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
from langchain.schema import SystemMessage

In [21]:
# DuckDuckGo 검색 도구
class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearch"
    description = "DuckDuckGo에서 검색을 수행합니다."
    search = DuckDuckGoSearchAPIWrapper()
    
    def _run(self, query: str) -> str:
        try:
            # max_results 파라미터 제거하고 기본 검색 사용
            results = self.search.run(query)
            if not results:
                return "DuckDuckGo 검색 결과를 찾을 수 없습니다."
            return results
        except Exception as e:
            return f"DuckDuckGo 검색 중 오류 발생: {str(e)}"

# Wikipedia 검색 도구
class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearch"
    description = "Wikipedia에서 검색을 수행합니다."
    
    def _run(self, query: str) -> str:
        url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json&origin=*"
        response = requests.get(url)
        data = response.json()
        
        if 'query' in data and 'search' in data['query']:
            results = data['query']['search']
            if results:
                return f"Wikipedia 검색 결과:\n{results[0]['snippet']}"
        return "Wikipedia에서 관련 정보를 찾을 수 없습니다."
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("WikipediaSearchTool does not support async")

# 웹 스크래핑 도구
class WebScrapingTool(BaseTool):
    name = "WebScraping"
    description = "웹사이트의 텍스트를 추출합니다."
    
    def _run(self, url: str) -> str:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 불필요한 요소 제거
            for script in soup(["script", "style"]):
                script.decompose()
            
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = ' '.join(chunk for chunk in chunks if chunk)
            
            return text[:2000]  # 텍스트 길이 제한
        except Exception as e:
            return f"웹 스크래핑 중 오류 발생: {str(e)}"
    
    def _arun(self, url: str) -> str:
        raise NotImplementedError("WebScrapingTool does not support async")

# 결과 저장 도구
class SaveResearchTool(BaseTool):
    name = "SaveResearch"
    description = "리서치 결과를 파일에 저장합니다. 입력은 저장할 전체 리서치 내용이어야 합니다."
    
    def _run(self, content: str) -> str:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"research_{timestamp}.txt"
        
        try:
            # 구조화된 형식으로 저장
            formatted_content = f"""
# XZ Backdoor 리서치 보고서
생성 시간: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

{content}

---
이 리서치는 AI 리서치 에이전트에 의해 생성되었습니다.
"""
            
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(formatted_content)
            return f"리서치 결과가 {filename}에 저장되었습니다."
        except Exception as e:
            return f"파일 저장 중 오류 발생: {str(e)}"

In [26]:

# 에이전트 초기화
llm = ChatOpenAI(temperature=0.1)

tools = [
    DuckDuckGoSearchTool(),
    WikipediaSearchTool(),
    WebScrapingTool(),
    SaveResearchTool()
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    max_iterations=10,  # 반복 제한 증가
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            당신은 전문적인 리서치 에이전트입니다. 주어진 주제에 대해 다음 순서로 리서치를 수행하세요:
            
            1. Wikipedia에서 기본 정보를 찾으세요.
            2. DuckDuckGo를 사용하여 추가 정보를 검색하세요.
            3. 관련 웹사이트의 콘텐츠를 스크래핑하세요.
            4. 수집된 모든 정보를 종합하여 구조화된 보고서를 작성하세요.
            5. SaveResearch 도구를 사용하여 결과를 파일에 저장하세요.
            
            보고서 작성 규칙:
            - 제목: "XZ Backdoor 리서치 보고서"
            - 섹션:
              1. 소개
              2. Wikipedia 정보
              3. 추가 정보
              4. 결론
              5. 참조 목록
            
            중요: 마지막 단계에서 반드시 SaveResearch 도구를 사용하여 결과를 저장하세요.
            저장하지 않으면 리서치가 완료되지 않은 것으로 간주됩니다.
            
            각 단계를 완료한 후에는 다음 단계로 진행하세요.
            모든 단계가 완료되면 반드시 SaveResearch 도구를 사용하여 결과를 저장하세요.
            """
        )
    }
)

In [27]:
# 에이전트 실행
try:
    query = "Research about the XZ backdoor"
    result = agent.invoke({"input": query})
    
    # 결과 출력
    print("\n리서치 결과:")
    print(result["output"])
    
    # 결과가 파일에 저장되었는지 확인
    if "리서치 결과가" in result["output"] and "에 저장되었습니다" in result["output"]:
        print("\n리서치 결과가 성공적으로 파일에 저장되었습니다.")
    else:
        print("\n리서치 결과 저장에 실패했습니다. 수동으로 저장을 시도합니다...")
        # 저장되지 않은 경우 수동으로 저장 시도
        save_tool = SaveResearchTool()
        save_result = save_tool.run(result["output"])
        print(save_result)
        
        # 저장된 파일 확인
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"research_{timestamp}.txt"
        if os.path.exists(filename):
            print(f"\n파일이 성공적으로 저장되었습니다: {filename}")
        else:
            print("\n파일 저장에 실패했습니다.")
        
except Exception as e:
    print(f"에이전트 실행 중 오류 발생: {str(e)}")



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `XZ backdoor`


Wikipedia 검색 결과:
In February 2024, a malicious <span class="searchmatch">backdoor</span> was introduced to the Linux build of the <span class="searchmatch">xz</span> utility within the liblzma library in versions 5.6.0 and 5.6.1 by an
Invoking: `WebScraping` with `https://en.wikipedia.org/wiki/XZ_backdoor`


XZ Utils backdoor - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us Contribute HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more ContributionsTalk Contents move to sidebar hide (Top) 1 Background 2 Mechanism 3 Response Toggle Response subsection 3.1 Remediation 3.2 Broader response 4 Notes 5 References 6 External links Toggle the table o